In [154]:
#Some useful packages

import pandas as pd
import numpy as np
import matplotlib.pylab as plt

In [167]:
#Load data and data reduction and data cleaning

df = pd.read_csv('liquors.csv')
df.drop(['Store Location', 'County', 'Invoice/Item Number', 'Item Description', 'Category Name', 'Vendor Name', 'Store Name', 'Address', 'City', 'Volume Sold (Gallons)', 'Zip Code'], axis=1, inplace=True)
df.drop(df.index[:10708015], inplace=True)
df.dropna(inplace=True)
df['State Bottle Retail'] = df['State Bottle Retail'].map(lambda x: x.lstrip('$'))
df['State Bottle Retail'] = df['State Bottle Retail'].map(lambda x: float(x))
df.head()

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Date,Store Number,County Number,Category,Vendor Number,Item Number,Pack,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters)
10708015,01/03/2017,4312,78.0,1012200.0,55.0,8208,6,1750,$10.52,15.78,3,$15.78,5.25
10708016,01/03/2017,4312,78.0,1042100.0,35.0,28206,12,750,$10.50,15.75,1,$189.00,0.75
10708017,01/03/2017,4312,78.0,1082000.0,259.0,65195,6,750,$9.83,14.75,1,$88.50,0.75
10708018,01/03/2017,4312,78.0,1081200.0,260.0,68037,12,1000,$16.50,24.75,2,$297.00,2.00
10708019,01/03/2017,4312,78.0,1012100.0,65.0,10627,12,1000,$9.71,14.57,1,$174.84,1.00


In [169]:
# Navie Bayes Classifier, part one: Scan data set:

sale_category = {}
sale_item = {}
category_item = {}
category_volume = {}
csv = {}
sv = {}


for i, r in df.iterrows():
    
    category = r['Category']
    item = r['Item Number']
    price = r['State Bottle Retail']
    volume = r['Volume Sold (Liters)']
    store = r['Store Number']
    vendor = r['Vendor Number']
    bottle = r['Bottles Sold']
    
    if category in sale_category:
        sale_category[category] += price*bottle
    else:
        sale_category[category] = price*bottle
        
    if item in sale_item:
        sale_item[item] += price*bottle
    else:
        sale_item[item] = price*bottle
        
    if category in category_item:
        if item in category_item[category]:
            category_item[category][item] += volume
        else:
            category_item[category][item] = volume
    else:
        category_item[category] = {}
        category_item[category][item] = volume
    
    if category in category_volume:
        category_volume[category] += volume
    else:
        category_volume[category] = volume
        
    if (category, store, vendor) in csv:
        csv[(category, store, vendor)] += 1
    else:
        csv[(category, store, vendor)] = 1
        
    if (store, vendor) in sv:
        sv[(store, vendor)] += 1
    else:
        sv[(store, vendor)] = 1

In [185]:
county_sale = {}
county_pop = {}
county_vendorV = {}
county_vendorR = {}
vendor_county = {}


for i, r in df.iterrows():
    
    county = r['County Number']
    v = r['Volume Sold (Liters)']
    p = r['State Bottle Retail']
    vendor = r['Vendor Number']
    b = r['Bottles Sold']
    category = r['Category']
    
    if county in county_sale:
        if category in county_sale[county]:
            county_sale[county][category] += p*b
        else:
            county_sale[county][category] = p*b
    else:
        county_sale[county] = {}
        county_sale[county][category] = p*b
    
    if county in county_pop:
        if category in county_pop[county]:
            county_pop[county][category] += v
        else:
            county_pop[county][category] = v
    else:
        county_pop[county] = {}
        county_pop[county][category] = v
    
    if county in county_vendorV:
        if vendor in county_vendorV[county]:
            county_vendorV[county][vendor] += v
        else:
            county_vendorV[county][vendor] = v
    else:
        county_vendorV[county] = {}
        county_vendorV[county][vendor] = v
    
    if county in county_vendorR:
        if vendor in county_vendorR[county]:
            county_vendorR[county][vendor] += p*b
        else:
            county_vendorR[county][vendor] = p*b
    else:
        county_vendorR[county] = {}
        county_vendorR[county][vendor] = p*b
    
    if vendor in vendor_county:
        if county in vendor_county[vendor]:
            vendor_county[vendor][county] += p*b
        else:
            vendor_county[vendor][county] = p*b
    else:
        vendor_county[vendor] = {}
        vendor_county[vendor][county] = p*b

In [184]:
def bayes_model(store, vendor):
    P = {}
    for key, value in csv.items():
        c, s, v = key
        if s==store and v==vendor:
            P[c] = value/sv[s,v]
    if (len(P)):
        return max(P, key=P.get)
    return None

In [189]:
def vendor_sale_county(vendor):
    return max(vendor_county[vendor], key=vendor_county[vendor].get)

In [190]:
def county_revenue(county):
    return max(county_sale[county], key=county_sale[county].get)

In [210]:
def county_vendorv(county):
    return max(county_vendorV[county], key=county_vendorV[county].get)

In [211]:
def county_vendorr(county):
    return max(county_vendorR[county], key=county_vendorR[county].get)

In [215]:
def county_p(county):
    return max(county_pop[county], key=county_pop[county].get)

In [212]:
vendor_sale_county(35)

77.0

In [213]:
county_revenue(78)

1081600.0

In [216]:
county_vendorr(77)

260.0

In [217]:
county_vendorv(77)

260.0